**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [1]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 6.6 MB/s 
     |████████████████████████████████| 1.0 MB 49.3 MB/s 
     |████████████████████████████████| 591 kB 32.4 MB/s 


**Import các thư viện cần thiết**

In [2]:
import os
import re
import math
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from underthesea import text_normalize
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

**Lấy dữ liệu các tài liệu/văn bản mẫu từ**: https://github.com/HUTECH-OpenCourseWare/IRS.git

In [3]:
!git clone https://github.com/HUTECH-OpenCourseWare/IRS.git

Cloning into 'IRS'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 271 (delta 0), reused 271 (delta 0), pack-reused 0
Receiving objects: 100% (271/271), 441.48 KiB | 1.74 MiB/s, done.


**Tiến hành thử nghiệm với danh sách các tài liệu/văn bản thuộc các chủ đề khác nhau**

In [4]:
# Chọn danh sách các chủ đề của tài liệu/văn bản cho thử nghiệm
topics = [
    'the-thao',
    'giao-duc',
    'khoa-hoc'
]

# Tạo một tập dữ liệu thử nghiệm gồm các tài liệu/văn bản thuộc về 2-3 chủ đề
# Cấu trúc dữ liệu dạng list - lưu thông tin danh sách các tài liệu/văn bản thuộc chủ đề khác nhau
# Mỗi tài liệu/văn bản sẽ tổ chức dạng 1 tuple với: (topic, nội_dung_văn_bản, danh_sách_token)
D = []

**Tiến hành viết một số hàm hỗ trợ cho việc đọc dữ liệu, xử lý và tách từ trong tiếng Việt.**

In [5]:
# Viết hàm tiền xử lý và tách từ tiếng Việt
def preprocess(doc):
  # Tiến hành xử lý các lỗi từ/câu, dấu câu, v.v. trong tiếng Việt với hàm text_normalize
  normalized_doc = text_normalize(doc)
  # Tiến hành tách từ
  tokens = word_tokenize(normalized_doc)
  # Tiến hành kết hợp các từ ghép trong tiếng Việt bằng '_'
  combined_tokens = [token.replace(' ', '_') for token in tokens]
  return (normalized_doc, combined_tokens)

# Viết hàm lấy danh sách các văn bản/tài liệu thuộc các chủ đề khác nhau
def fetch_doc_by_topic(topic):
  data_root_dir_path = '/content/IRS/data/vnexpress/{}'.format(topic)
  docs = []
  for file_name in os.listdir(data_root_dir_path):
    file_path = os.path.join(data_root_dir_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
      lines = []
      for line in f:
        line = line.lower().strip()
        lines.append(line)
    doc = " ".join(lines)
    clean_doc = re.sub('\W+',' ', doc)
    (normalized_doc, tokens) = preprocess(clean_doc)
    docs.append((topic, normalized_doc, tokens))
  return docs

**Tiến hành tạo tập dữ liệu thử nghiệm với các tài liệu/văn bản thuộc danh sách chủ đề [topics] đã lựa chọn bên trên**

In [6]:
# Cấu trúc dữ liệu dictionary để lưu thông tin chủ đề-tài liệu, nhằm hỗ trợ cho việc tìm kiếm nhanh
topic_doc_idxes_dict = {}
doc_idx_topic_dict = {}

# Duyệt qua từng chủ đề
doc_idx = 0
for topic in topics:
  current_topic_docs = fetch_doc_by_topic(topic)
  topic_doc_idxes_dict[topic] = []
  for (topic, normalized_doc, tokens) in current_topic_docs:
    topic_doc_idxes_dict[topic].append(doc_idx)
    doc_idx_topic_dict[doc_idx] = topic
    doc_idx+=1
  D += current_topic_docs

doc_size = len(D)

print('Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [{}]'.format(doc_size))
for topic in topic_doc_idxes_dict.keys():
  print(' - Chủ đề [{}] có [{}] tài liệu/văn bản.'.format(topic, len(topic_doc_idxes_dict[topic])))

Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [109]
 - Chủ đề [the-thao] có [39] tài liệu/văn bản.
 - Chủ đề [giao-duc] có [35] tài liệu/văn bản.
 - Chủ đề [khoa-hoc] có [35] tài liệu/văn bản.


**Tiến hành biến đổi các tài liệu/văn bản trong tập (D) về dạng các TF-IDF vectors - trong bài thực hành này chúng ta sẽ sử dụng thư viện Scikit-Learn (TfidfVectorizer) https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html**

In [7]:
# Khởi tạo đối tượng TfidfVectorizer
vectorizer = TfidfVectorizer()

# Chúng ta sẽ tạo ra một tập danh sách các tài liệu/văn bản dạng list đơn giản để thư viện Scikit-Learn có thể đọc được
sk_docs = []

# Duyệt qua từng tài liệu/văn bản có trong (D)
for (topic, normalized_doc, tokens) in D:
  # Chúng ta sẽ nối các từ/tokens đã được tách để làm thành một văn bản hoàn chỉnh
  text = ' '.join(tokens)
  sk_docs.append(text)

# Tiến hành chuyển đổi các tài liệu/văn bản về dạng các TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(sk_docs)

# Chuyển ma trận tfidf_matrix từ dạng cấu trúc thưa sang dạng đầy đủ để thuận tiện cho việc tính toán
tfidf_matrix = tfidf_matrix.todense()

**QUÁ TRÌNH PHÂN LỚP: với phương pháp phân lớp dựa trên kNN chúng ta sẽ không cần đến quá trình huấn luyện mô hình mà sẽ trực tiếp dùng tập dữ liệu huấn luyện để xác định chủ đề/lớp cho văn bản kiểm thử (mới)**

In [8]:
# Xác định số lượng hàng xóm lân cận (k) - giá trị (k) nên là số lẻ
k = 5

# Viết hàm giúp chuyển đổi một văn bản mới về dạng tfidf vector
def parse_text(text):
  (normalized_doc, combined_tokens) = preprocess(text)
  parsed_text = ' '.join(combined_tokens)
  text_tfidf_vector = vectorizer.transform([parsed_text])[0].todense()
  return text_tfidf_vector

test_doc = 'Trừ La Liga, bốn giải lớn còn lại của châu Âu đều thi đấu cuối tuần này trước khi các CLB nhả người cho đội tuyển dịp World Cup 2022. Man City đóng góp năm cầu thủ trong danh sách tuyển Anh dự World Cup 2022, nhiều nhất trong số các CLB Ngoại hạng Anh. Dựa vào những gì các học trò Pep Guardiola đang thể hiện, đó không phải bất ngờ. Nhưng HLV Brentford Thomas Frank cho rằng học trò của ông, tiền đạo Ivan Toney, xứng đáng không kém.'

test_doc_tfidf_vector = parse_text(test_doc)

# Chúng ta có một cấu trúc dictionary để lưu trữ mức độ tương đồng 
# giữa tài liệu/văn bản kiểm thử & các tài liệu/văn bản trong tập huấn luyện
doc_idx_test_doc_sim_dict = {}

print('Xác định mức độ tương đồng giữa tập tài liệu/văn bản huấn luyện và tài liệu/văn bản kiểm thử...')
for doc_idx, doc_tfidf_vector in enumerate(tfidf_matrix):
  cs_sim = 1 - distance.cosine(doc_tfidf_vector, test_doc_tfidf_vector)
  doc_idx_test_doc_sim_dict[doc_idx] = cs_sim

# Sắp xếp lại thứ tự các tài liệu/văn bản theo mức độ tương đồng (giảm dần)
sorted_doc_idx_test_doc_sim_dict = sorted(doc_idx_test_doc_sim_dict.items(), key=lambda item: item[1], reverse=True)

# Lấy ra top-k tài liệu/văn bản gần nhất với tài liệu/văn bản kiểm thử [test_doc_tfidf_vector]
top_k_nearest_neighors = sorted_doc_idx_test_doc_sim_dict[:k]

# Chúng ta có một dictionary để thống kê số lượng các chủ đề của các tài liệu/văn bản gần nhất với [test_doc_tfidf_vector]
top_k_topic_dict = {}

for (doc_idx, cs_sim) in top_k_nearest_neighors:
  belonged_topic = doc_idx_topic_dict[doc_idx]
  print(f' - Tài liệu: [{doc_idx}] - chủ đề: [{belonged_topic}], có độ tương đồng: [{cs_sim:.6f}]')
  if belonged_topic not in top_k_topic_dict.keys():
    top_k_topic_dict[belonged_topic] = 1
  else:
    top_k_topic_dict[belonged_topic] += 1

# Sắp xếp lại theo thứ tự giảm dần của số lượng chủ đề xuất hiện
sorted_top_k_topic_dict = sorted(top_k_topic_dict.items(), key=lambda item: item[1], reverse=True)

# Xác định chủ đề/lớp của [test_doc_tfidf_vector] chính là chủ đề/lớp xuất hiện nhiều nhất
print(f'Chủ đề/lớp của tài liệu/văn bản được xác định là: [{sorted_top_k_topic_dict[0][0]}]')

Xác định mức độ tương đồng giữa tập tài liệu/văn bản huấn luyện và tài liệu/văn bản kiểm thử...
 - Tài liệu: [5] - chủ đề: [the-thao], có độ tương đồng: [0.226069]
 - Tài liệu: [1] - chủ đề: [the-thao], có độ tương đồng: [0.185795]
 - Tài liệu: [11] - chủ đề: [the-thao], có độ tương đồng: [0.184284]
 - Tài liệu: [29] - chủ đề: [the-thao], có độ tương đồng: [0.184202]
 - Tài liệu: [15] - chủ đề: [the-thao], có độ tương đồng: [0.158986]
Chủ đề/lớp của tài liệu/văn bản được xác định là: [the-thao]
